In [1]:
import os
import pandas as pd
import csv
import re

In [11]:
def match_label(to_match, labels):
    match_regex = re.compile(f".*({to_match}).*", flags=re.IGNORECASE)
    desired_regex = re.compile(r".*(interest|desire|prefer|open).*", flags=re.IGNORECASE)
    matched_label = None
    desired_label = None
    for label in labels:
        if match_regex.match(label):
            if desired_regex.match(label):
                desired_label = label
            else:
                matched_label = label
    if matched_label is None and desired_label:
        matched_label = desired_label
    return matched_label

In [15]:
def detect_interested_label(labels: list[str]):
    """ 
    Detect labels of interested, i.e ["name or email (as ID)", "title/role", "department/area/function", "location"]
    """
    title_label = match_label("title|role|position|domain", labels)
    function_label = match_label("department|area|function|team|discipline", labels)
    city_label = match_label("city", labels)
    country_label = match_label("country", labels)
    location_label = match_label("location", labels)

    # title_regex = re.compile(r".*(title|role|position|domain).*", flags=re.IGNORECASE)
    # function_regex = re.compile(r".*(department|area|function|team|discipline).*", flags=re.IGNORECASE)
    # location_regex = re.compile(r".*(location).*", flags=re.IGNORECASE)
    # city_regex = re.compile(r".*(city).*", flags=re.IGNORECASE)
    # country_regex = re.compile(r".*(country).*", flags=re.IGNORECASE)
    # desired_label_regex = re.compile(r".*(interest|desire|prefer|open).*", flags=re.IGNORECASE)
    # title_label = None
    # desired_title_label = None
    # function_label = None
    # desired_function_label = None
    # location_label = None
    # desired_location_label = None
    # label_map = {}
    # for label in labels:
    #     if title_regex.match(label):
    #         if desired_label_regex.match(label):
    #             desired_title_label = label
    #         else:
    #             title_label = label
    #     elif function_regex.match(label):
    #         if desired_label_regex.match(label):
    #             desired_function_label = label
    #         else:
    #             function_label = label
    #     elif city_regex.match(label):
    #         if desired_label_regex.match(label):
    #             desired_location_label = label
    #         else:
    #             location_label = label
    #         # possible_title_labels.append(label)
    #         # print(f"possible title lable: {label}")
    # if title_label is None and desired_title_label:
    #     title_label = desired_title_label
    # if function_label is None and desired_function_label:
    #     function_label = desired_function_label
    # if location_label is None and desired_location_label:
    #     location_label = desired_location_label
    location_label = city_label or country_label or location_label
    label_list = [title_label, function_label, location_label]
    return label_list
    # label_map = {
    #     "title": title_label,
    #     "function": function_label,
    #     "location": location_label
    # }
    # return label_map

In [16]:
# Create an empty list to store the first row of each CSV file
first_rows = []

# if the first column more than 50 character it is not likely to be column
len_character_threshold = 50

# Define a regular expression pattern to match HTML tags
html_tag_pattern = re.compile(r'<\s*(html|head|body|table)\b[^>]*>', re.IGNORECASE)

invalid_first_row_count = 0
invalid_file_count = 0
employee_count = 0

def clean_csv(input_file, output_dir):
    rows = []
    labels = []
    # print(f"Opening file: {input_file}")
    # Open the CSV file
    with open(input_file, 'r', encoding='utf-8') as csvfile:
        # Read the entire contents of the CSV file as a string
        file_content = csvfile.read()
        
        # Check if the file contains any HTML tags
        if re.search(html_tag_pattern, file_content):
            # Skip to the next file if the file contains HTML content
            print(f"skipping: HTML element found in file {input_file}")
            return None
        
        csvfile.seek(0)  # move file pointer to start of file
        # Create a CSV reader object
        reader = csv.reader(csvfile, delimiter=',')
        try:
            # Get the first row of the CSV file
            while len(labels) == 0:
                current_row = next(reader)
                if (len(current_row[0]) >= len_character_threshold) \
                        or (len(current_row) > 1 and len(current_row[1]) >= len_character_threshold):
                    # print("skipping: too many characters in first row cell")
                    # print(current_row[0])
                    continue

                row_text = "|".join(current_row).lower()
                if row_text.find("name") != -1 or \
                    row_text.find("nome") != -1 or \
                    row_text.find("nombre") != -1:
                    labels = current_row

            print(f"label row found: {labels}")
            interested_labels = ["||".join(labels)]
            interested_labels.extend(detect_interested_label(labels))
            return interested_labels
        
            while False:
                row = next(reader)
                if len(row) <= len(labels):
                    if "".join(row).strip() == "":
                        print("skipping empty row")
                        continue
                    rows.append(row)
                else:
                    rows.append(row[:len(labels)])
        except StopIteration as e:
            print(f"Reached end of file: {input_file}")
            # Skip this file if it has less than one row
            if len(labels) > 0 and len(rows) > 0:
                df = pd.DataFrame(rows, columns=labels)
                output_file = input_file.split('/')[-1].split('.')[0]
                output_path = f"{output_dir}/{output_file}-cleaned.csv"
                print(f"writing output file to {output_path}")
                df.to_csv(output_path)
                return output_path
            return None
        except Exception as e:
            print(f"Exception error file not valid: {e} in {input_file}")
            return None

In [17]:
folder_path = "../../employee_csv_20230322/"
label_data = []
invalid_files = []
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        interested_labels = clean_csv(input_file=f"{folder_path}/{filename}", output_dir="output")
        d = [filename]
        if interested_labels is not None:
            d.extend(interested_labels)
            label_data.append(d)
        else:
            invalid_files.append(filename)
        #     d.extend(["", ""])
        #     label_data.append()
label_df = pd.DataFrame(data=label_data, columns=["file_name", "labels", "title_label", "function_label", "location_label"])
print(label_df.shape)
print(label_df[label_df["function_label"].isna()].shape)

label row found: ['Name', 'Previous Title', 'Current Location', 'Preferred Work Mode', 'LinkedIn Page', 'Last updated: Wednesday, 12/7/22 @ 2:12pm ET']
label row found: ['Nombre y apellido', 'Posición', 'Linkedin', 'Email', 'Teléfono', 'Link CV']
label row found: ['NOME', 'CARGO', 'ÁREA', 'LINKEDIN']
label row found: ['Timestamp', 'Email Address', 'What is your first name?', 'What is your last name?', 'What was your role/title at Stitch Fix?', 'What role(s) are you interested in?', 'Please share your LinkedIn profile.', 'Where are you located?', 'What is your Work Mode preference (e.g. site-based, remote, etc)?', 'What is your personal email address?', 'Which team were you on at Stitch Fix?']
label row found: ['Full Name', 'Title', 'Email', 'LinkedIn URL']
label row found: ['Name ', 'Function', 'Title / Role', 'LinkedIn URL', 'Personal email address', 'Location', 'Work preference', 'Roles / Areas of interest']
label row found: ['', 'Update my info/status', 'Name', 'Role / Position', 'D